In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from bench_lib.evaluation import compute_agreement_score, load_human_labels
from bench_lib.utils import enable_info_logs
import pandas as pd

enable_info_logs()

human_labels, questions, comment_cols = load_human_labels()
human_labels_long = pd.melt(
    human_labels, id_vars=["post_id", "classification_by"], value_vars=questions
)
human_consistency_df = compute_agreement_score(human_labels_long)
human_consistency_df.head(2)

from bench_lib.evaluation import benchmark_results_folder, check_self_consistency


folder = "self-consistency"
csv = benchmark_results_folder() / folder / "model_labels.csv"
flip_counts_all, consistency_score = check_self_consistency(
    csv_paths=[str(csv)],
    model_to_check="google/gemma-3-27b-it",
    n_runs=5,
)

In [ ]:
from bench_lib.evaluation import load_ai_labels

folder = "self-consistency"
ai_labels_long = load_ai_labels([folder], questions, comment_cols)
ai_labels_long.head(2)

In [ ]:
from bench_lib.evaluation import agreement_score


ai_agreement_df = compute_agreement_score(
    ai_labels_long, groupby=["post_id", "Model ID", "variable"]
)
ai_agreement_df.head(2)

In [ ]:
import pandas as pd

joint_consistency = pd.merge(
    ai_agreement_df,
    human_consistency_df,
    on=["post_id", "variable"],
    how="left",
    suffixes=("_ai", "_human"),
)
joint_consistency.head(2)